<a href="https://colab.research.google.com/github/kodebot/classical_tamil_llm/blob/main/Fine_tune_Tamil_LLaMA_7b_instruct_for_Tamil_Poem_v0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune Tamil Llama 7b instruct for Tamil Poem


In [5]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.32.0 trl==0.4.7

In [6]:
import os
import torch
from datasets import load_dataset,concatenate_datasets
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [7]:
# The model that you want to train from the Hugging Face hub
model_name = "abhinand/tamil-llama-7b-instruct-v0.2"

# The instruction dataset to use
dataset_names = [
    "kodebot/Thirukural_tamil_with_meaning",
    "kodebot/Agananuru_Tamil_with_meaning",
    "kodebot/Purananuru_Tamil_with_meaning"
]

# Fine-tuned model name
new_model = "tamil-llama-7b-instruct-tamil-poem-fine-tune-experiment-v0.1"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [9]:
# Load dataset (you can process it here)
dataset_1 = load_dataset(dataset_names[0], split="train")
dataset_split_1 = dataset_1.train_test_split(test_size=0.1)

dataset_2 = load_dataset(dataset_names[1], split="train")
dataset_split_2 = dataset_2.train_test_split(test_size=0.1)

dataset_3 = load_dataset(dataset_names[2], split="train")
dataset_split_3 = dataset_2.train_test_split(test_size=0.1)

dataset = concatenate_datasets([dataset_1, dataset_2, dataset_3])
dataset_train = concatenate_datasets([dataset_split_1['train'], dataset_split_2['train'], dataset_split_3['train']])
dataset_test = concatenate_datasets([dataset_split_1['test'], dataset_split_2['test'], dataset_split_3['test']])

dataset_train.save_to_disk("./dataset_train")
dataset_test.save_to_disk("./dataset_test")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)


Saving the dataset (0/1 shards):   0%|          | 0/1917 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/213 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

ValueError: Your setup doesn't support bf16/gpu. You need torch>=1.10, using Ampere GPU with cuda>=11.0

In [10]:

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=False,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_train,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1917 [00:00<?, ? examples/s]

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,5.790100
50,5.011300
75,5.055200
100,4.621900
125,4.849600
150,4.496000
175,4.769500
200,4.415200
225,4.691600
250,4.393700


In [12]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

generated_texts = []
reference_texts = []
for inst in dataset_test:
  # Run text generation pipeline with our next model
  splits = inst["text"].replace("<s>", "").replace("</s>", "").replace("[INST]", "").split("[/INST]")
  prompt = splits[0].strip()
  reference_texts.append(splits[1].strip())
  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
  result = pipe(f"<s>[INST] {prompt} [/INST]")
  generated_texts.append(result[0]['generated_text'].replace("<s>", "").replace("</s>", "").replace("[INST]", "").split("[/INST]")[1])
  print(len(generated_texts), " of ", 10)
  if len(generated_texts) >= 10:
    break

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


1  of  10
2  of  10
3  of  10
4  of  10
5  of  10
6  of  10
7  of  10
8  of  10
9  of  10
10  of  10


In [13]:
reference_texts

['குறைகூறுவோறின் சொற்களைக் செவிகைக்கும் நிலையிலும் பொறுக்கின்ற பண்பும் உடைய அரசனது குடைநிழலில் உலகம் தங்கும்.',
 '(அருள் இல்லாதவன் ) தன்னை விட மெலிந்தவர் மேல் துன்புறுத்த செல்லும் போது, தன்னை விட வலியவரின் முன் தான் அஞ்சி நிற்கும் நிலைமையை நினைக்க வேண்டும்.',
 'இந்தச் செயலை இக்கருவியால் இன்னவன் செய்துமுடிப்பான் என்று ஆராய்ந்த பிறகே அத் தொழிலை அவனிடம் ஒப்படைக்க வேண்டும்.',
 'புகழ் பெறாமல் வாழ்வைக் கழித்தவருடைய உடம்பைச் சுமந்த நிலம், வசையற்ற வளமான பயனாகிய விளைவு இல்லாமல் குன்றிவிடும்.',
 'பொருந்து போதெல்லாம் உயிர் தளிர்க்கும் படியாகத் தீண்டுதலால் இவளுக்கு தோள்கள் அமிழ்தத்தால் செய்யப்பட்டிருக்க வேண்டும்.',
 'ஒருவன் மற்றவர் நாணத்தக்க பழிக்குக் காரணமாக இருந்தும் தான் நாணாமலிருப்பானானால், அறம் நாணி அவனைக் கைவிடும் தன்மையுடையதாகும்.',
 'நேராகத் தோன்றினும் அம்பு கொடியது; வளைவுடன் தோன்றினாலும் யாழின் கொம்பு நன்மையானது. மக்களின் பண்புகளையும் செயல்வகையால் உணர்ந்துகொள்ள வேண்டும்.',
 'நோயில்லாதிருத்தல், செல்வம், விளை பொருள், வளம், இன்பவாழ்வு, நல்ல காவல் இந்த ஐந்தும் நாட்டிற்கு அழகு என்று கூறுவர்.',

In [14]:
generated_texts

[' (இறப்பு நிகழும் முன்) இன்பம் தருகின்ற பண்புகளை உடைய வேந்தன், தன் தலைவியின் அழகு, அவன் தலைவியின் அழகுக்கு நிகரானது என்று கூறினால், அந்த அழகுக்குக் குறையாக இருக்கும். இந்த இரண்டு அழகுக்கும் இடையே உள்ள வேறுபாட்டைத் தன்னால் தாங்கிக் கொள்ள முடியாது. இந்த இரண்டு அழகுக்கும் இடையே உள்ள வேறுபாட்டைத் தன்னால் தாங்கிக் கொள்ள முடியாது.',
 ' தனக்குப் பொருந்தியவரிடம் தன்னை நினைக்க வேண்டும், அதற்கு அப்பொழுது தான் தன்னிடம் உள்ளவர் தனது எண்ணத்தை நினைப்பார். அது போலவே, தன்னிடம் உள்ளவரைத் தன் எண்ணத்தை நினைக்கும். இந்த இரண்டு நிலைகளையும் ஒருவர் தன்னிடம் உள்ளவர் தனக்கு நினைக்கும் நிலை என்று கொள்ளலாம். இந்த இரண்டு நிலைகளும் ஒன்றா அல்லது வேறு வேறுவா என்பது கேள்வி.',
 ' இந்த காரியத்தைப் பற்றி உனக்குத் தெரிந்தால், இந்த விஷயத்திற்கு இந்த வழியில் தான் காரணம் என்று நினைக்காதே. அதுவே அவனுக்கு துன்பம் தரும்.',
 ' யான் வண்பானின் குறும் ஒலிக்கு ஒப்பானது, யாக்கை வைத்த நிலம், வண்பானின் குறும் ஒலிக்கு ஒப்பானது. அது யான் வண்பானின் குறும் ஒலிக்கு ஒப்பானது. இது அகநானூறு பாடல் 192  இந்த பாடலின் பொருள்: யாக்கு நிலம், யான் 

In [15]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [17]:
!pip install evaluate rouge_score

In [18]:
import evaluate

In [19]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=generated_texts,
                         references=reference_texts)
print(results)

{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}


In [20]:
# Empty VRAM
del model
del pipe
del trainer


In [21]:

import gc
gc.collect()
gc.collect()

0

In [22]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location=map

In [23]:
from huggingface_hub import notebook_login
notebook_login()


In [24]:

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.83G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kodebot/tamil-llama-7b-instruct-tamil-poem-fine-tune-experiment-v0.1/commit/0111c8bf2cee43b61380e441f24f093708f5269c', commit_message='Upload tokenizer', commit_description='', oid='0111c8bf2cee43b61380e441f24f093708f5269c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/kodebot/tamil-llama-7b-instruct-tamil-poem-fine-tune-experiment-v0.1', endpoint='https://huggingface.co', repo_type='model', repo_id='kodebot/tamil-llama-7b-instruct-tamil-poem-fine-tune-experiment-v0.1'), pr_revision=None, pr_num=None)